In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index


array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class SimpleDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

    def __len__(self):
        return len(self.X)


class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim=32):
        super(Encoder, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, embedding_dim)
        )

    def forward(self, x):
        return self.fc(x)

batch_size = 512

x_test_adv_0_1 = np.array(x_test_adv_0_1)
y_label_0_1 = np.array(y_label_0_1)
train_dataset = SimpleDataset(x_test_adv_0_1, y_label_0_1)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

input_dim = x_test_adv_0_1.shape[1]
embedding_dim = 32
net = Encoder(input_dim, embedding_dim).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)



In [6]:
num_epochs = 20

for epoch in range(num_epochs):
    net.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = net(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_loss:.4f}")

net.eval()
def get_embeddings(net, data_loader):
    all_embeddings = []
    all_labels = []
    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch = X_batch.to(device)
            embeddings = net(X_batch)
            all_embeddings.append(embeddings.cpu())
            all_labels.append(y_batch)
    return torch.cat(all_embeddings, dim=0), torch.cat(all_labels, dim=0)

train_embeddings, train_labels = get_embeddings(net, train_loader)


Epoch 1/20, Training Loss: 0.5018
Epoch 2/20, Training Loss: 0.3463
Epoch 3/20, Training Loss: 0.3145
Epoch 4/20, Training Loss: 0.2953
Epoch 5/20, Training Loss: 0.2819
Epoch 6/20, Training Loss: 0.2715
Epoch 7/20, Training Loss: 0.2631
Epoch 8/20, Training Loss: 0.2567
Epoch 9/20, Training Loss: 0.2508
Epoch 10/20, Training Loss: 0.2459
Epoch 11/20, Training Loss: 0.2420
Epoch 12/20, Training Loss: 0.2383
Epoch 13/20, Training Loss: 0.2346
Epoch 14/20, Training Loss: 0.2324
Epoch 15/20, Training Loss: 0.2302
Epoch 16/20, Training Loss: 0.2277
Epoch 17/20, Training Loss: 0.2262
Epoch 18/20, Training Loss: 0.2247
Epoch 19/20, Training Loss: 0.2228
Epoch 20/20, Training Loss: 0.2215


In [7]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]



In [8]:
def predict_by_nearest_Euclidean(train_embeddings, train_labels, test_embeddings, device='cpu', batch_size=1024):
    train_embeddings = train_embeddings.to(device)
    test_embeddings = test_embeddings.to(device)
    train_labels = train_labels.to(device)

    preds = []

    for i in range(0, test_embeddings.size(0), batch_size):
        batch_test = test_embeddings[i:i+batch_size]  # (batch_size, D)

        train_norm = (train_embeddings ** 2).sum(1).reshape(1, -1)  # (1, N_train)
        test_norm = (batch_test ** 2).sum(1).reshape(-1, 1)          # (batch_test, 1)
        inner_product = torch.matmul(batch_test, train_embeddings.T)  # (batch_test, N_train)
        dists = test_norm + train_norm - 2 * inner_product

        nearest_idx = torch.argmin(dists, dim=1)  # (batch_size,)
        batch_preds = train_labels[nearest_idx]
        preds.append(batch_preds.cpu())

    preds = torch.cat(preds, dim=0)
    return preds.numpy()



In [14]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    X = X.astype(np.float32)
    test_dataset = SimpleDataset(X, y)
    test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)
    test_embeddings, test_labels = get_embeddings(net, test_loader)
    y_pred = predict_by_nearest_Euclidean(train_embeddings, train_labels, test_embeddings, device=device, batch_size=1024)



    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/Recommendation_Euclidean/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/Recommendation_Euclidean/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [28289   312  2274  1323   140  3465     1   319 77442   287   119]
(28289, 56) (28289,)
Save 0 to baseline_Def1.npy
(312, 56) (312,)
Save 1 to baseline_Def2.npy
(2274, 56) (2274,)
Save 2 to baseline_Def3.npy
(1323, 56) (1323,)
Save 3 to baseline_Def4.npy
(140, 56) (140,)
Save 4 to baseline_Def5.npy
(3465, 56) (3465,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(319, 56) (319,)
Save 7 to baseline_Def8.npy
(77442, 56) (77442,)
Save 8 to baseline_Def9.npy
(287, 56) (287,)
Save 9 to baseline_Def10.npy
(119, 56) (119,)
Save 10 to baseline_Def11.npy
Execution Time: 4.791837 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [219880    356   2589   3730    676  30697      1   2162  80120   1059
    643]
(219880, 56) (219880,)
Save 0 to BIM_Def1.npy
(356, 56) (356,)
Save 1 to BIM_Def2.npy
(2589, 56) (2589,)
Save 2 to BIM_Def3.npy
(3730, 56) (3730,)
Save 3 to BIM_Def4.npy
(676, 56) (676,)
Save 4 to BIM_Def5.npy
(30697, 56)

In [16]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []


for attack_name in attack_names:
    each = []
    num = 0
    each.append(f"Recommendation")
    for i in range(1, 12):
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/Recommendation_Euclidean/x_test_adv_{attack_name}_Def{i}.npy"
        try:
            y = np.load(y_filename)
            num+=y.shape[0]
            each.append(y.shape[0])
        except FileNotFoundError:
            each.append(0)
    each.append(num)
    all_attack_list.append(each)

import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Recommendation_Euclidean.csv")



In [9]:
def predict_by_nearest_cosine(train_embeddings, train_labels, test_embeddings, device='cpu', batch_size=1024):
    train_embeddings = train_embeddings.to(device)
    test_embeddings = test_embeddings.to(device)
    train_labels = train_labels.to(device)

    train_embeddings = train_embeddings / torch.norm(train_embeddings, dim=1, keepdim=True)
    test_embeddings = test_embeddings / torch.norm(test_embeddings, dim=1, keepdim=True)

    preds = []

    for i in range(0, test_embeddings.size(0), batch_size):
        batch_test = test_embeddings[i:i+batch_size]  # (batch_size, D)

        cosine_sim = torch.matmul(batch_test, train_embeddings.T)  # (batch_test, N_train)
        cosine_dist = 1 - cosine_sim

        nearest_idx = torch.argmin(cosine_dist, dim=1)  # (batch_size,)
        batch_preds = train_labels[nearest_idx]
        preds.append(batch_preds.cpu())

    preds = torch.cat(preds, dim=0)
    return preds.numpy()

In [10]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    X = X.astype(np.float32)
    test_dataset = SimpleDataset(X, y)
    test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)
    test_embeddings, test_labels = get_embeddings(net, test_loader)
    y_pred = predict_by_nearest_cosine(train_embeddings, train_labels, test_embeddings, device=device, batch_size=1024)



    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/Recommendation_cosine/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/Recommendation_cosine/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [26852   289  2227  1309   146  3496     1   321 78905   316   109]
(26852, 56) (26852,)
Save 0 to baseline_Def1.npy
(289, 56) (289,)
Save 1 to baseline_Def2.npy
(2227, 56) (2227,)
Save 2 to baseline_Def3.npy
(1309, 56) (1309,)
Save 3 to baseline_Def4.npy
(146, 56) (146,)
Save 4 to baseline_Def5.npy
(3496, 56) (3496,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(321, 56) (321,)
Save 7 to baseline_Def8.npy
(78905, 56) (78905,)
Save 8 to baseline_Def9.npy
(316, 56) (316,)
Save 9 to baseline_Def10.npy
(109, 56) (109,)
Save 10 to baseline_Def11.npy
Execution Time: 5.211244 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [239147    568   2542   4913   1070  11799      2    588  80768    400
    116]
(239147, 56) (239147,)
Save 0 to BIM_Def1.npy
(568, 56) (568,)
Save 1 to BIM_Def2.npy
(2542, 56) (2542,)
Save 2 to BIM_Def3.npy
(4913, 56) (4913,)
Save 3 to BIM_Def4.npy
(1070, 56) (1070,)
Save 4 to BIM_Def5.npy
(11799, 5

In [21]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []


for attack_name in attack_names:
    each = []
    num = 0
    each.append(f"Recommendation")
    for i in range(1, 12):
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/Recommendation_cosine/x_test_adv_{attack_name}_Def{i}.npy"
        try:
            y = np.load(y_filename)
            num+=y.shape[0]
            each.append(y.shape[0])
        except FileNotFoundError:
            each.append(0)
    each.append(num)
    all_attack_list.append(each)

import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Recommendation_cosine.csv")
all_attack_list


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Recommendation,26852,289,2227,1309,146,3496,1,321,78905,316,109,113971
1,Recommendation,239147,568,2542,4913,1070,11799,2,588,80768,400,116,341913
2,Recommendation,236796,334,2300,5615,819,15033,1,759,79594,417,245,341913
3,Recommendation,239147,568,2542,4913,1070,11799,2,588,80768,400,116,341913
4,Recommendation,73848,736,2579,443,433,239146,2703,18998,2443,355,229,341913
5,Recommendation,232649,980,5576,6841,1096,8443,7,1074,84276,627,344,341913
6,Recommendation,68745,888,7152,4068,433,9176,6,963,249290,871,321,341913
7,Recommendation,241421,196,501,2118,581,20788,9,530,71806,3029,934,341913
8,Recommendation,236918,681,2633,6138,1935,12163,1,468,80417,418,141,341913
9,Recommendation,239364,571,2579,6369,1785,10112,1,755,79853,396,128,341913


In [17]:
def predict_by_nearest_manhattan(train_embeddings, train_labels, test_embeddings, device='cpu', batch_size=2048):
    train_embeddings = train_embeddings.to(device)
    test_embeddings = test_embeddings.to(device)
    train_labels = train_labels.to(device)

    preds = []

    for i in range(0, test_embeddings.size(0), batch_size):
        batch_test = test_embeddings[i:i+batch_size]  # (batch_size, D)

        batch_size_actual = batch_test.size(0)
        dists = torch.zeros(batch_size_actual, train_embeddings.size(0), device=device)
        for j in range(0, train_embeddings.size(0), batch_size):
            batch_train = train_embeddings[j:j+batch_size]
            abs_diff = torch.abs(batch_test.unsqueeze(1) - batch_train.unsqueeze(0))
            dists[:, j:j+batch_size] = abs_diff.sum(dim=2)

        nearest_idx = torch.argmin(dists, dim=1)  # (batch_size,)
        batch_preds = train_labels[nearest_idx]
        preds.append(batch_preds.cpu())

    preds = torch.cat(preds, dim=0)
    return preds.numpy()

In [18]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    X = X.astype(np.float32)
    test_dataset = SimpleDataset(X, y)
    test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)
    test_embeddings, test_labels = get_embeddings(net, test_loader)
    y_pred = predict_by_nearest_manhattan(train_embeddings, train_labels, test_embeddings, device=device, batch_size=1024)



    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/Recommendation_manhattan/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/Recommendation_manhattan/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [26795   289  2248  1297   144  3422     1   325 79011   327   112]
(26795, 56) (26795,)
Save 0 to baseline_Def1.npy
(289, 56) (289,)
Save 1 to baseline_Def2.npy
(2248, 56) (2248,)
Save 2 to baseline_Def3.npy
(1297, 56) (1297,)
Save 3 to baseline_Def4.npy
(144, 56) (144,)
Save 4 to baseline_Def5.npy
(3422, 56) (3422,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(325, 56) (325,)
Save 7 to baseline_Def8.npy
(79011, 56) (79011,)
Save 8 to baseline_Def9.npy
(327, 56) (327,)
Save 9 to baseline_Def10.npy
(112, 56) (112,)
Save 10 to baseline_Def11.npy
Execution Time: 80.719233 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [220692    566   2311   5889    679  30343      3    836  80072    392
    130]
(220692, 56) (220692,)
Save 0 to BIM_Def1.npy
(566, 56) (566,)
Save 1 to BIM_Def2.npy
(2311, 56) (2311,)
Save 2 to BIM_Def3.npy
(5889, 56) (5889,)
Save 3 to BIM_Def4.npy
(679, 56) (679,)
Save 4 to BIM_Def5.npy
(30343, 56

In [20]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []


for attack_name in attack_names:
    each = []
    num = 0
    each.append(f"Recommendation_manhattan")
    for i in range(1, 12):
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/Recommendation_manhattan/x_test_adv_{attack_name}_Def{i}.npy"
        try:
            y = np.load(y_filename)
            num+=y.shape[0]
            each.append(y.shape[0])
        except FileNotFoundError:
            each.append(0)
    each.append(num)
    all_attack_list.append(each)

import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Recommendation_manhattan.csv")
all_attack_list


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Recommendation_manhattan,26795,289,2248,1297,144,3422,1,325,79011,327,112,113971
1,Recommendation_manhattan,220692,566,2311,5889,679,30343,3,836,80072,392,130,341913
2,Recommendation_manhattan,222976,352,2301,4947,469,29419,1,832,80015,387,214,341913
3,Recommendation_manhattan,220692,566,2311,5889,679,30343,3,836,80072,392,130,341913
4,Recommendation_manhattan,75074,681,2439,394,457,240595,1021,18123,2667,263,199,341913
5,Recommendation_manhattan,216271,980,5473,7690,896,23789,1,1322,84468,674,349,341913
6,Recommendation_manhattan,68752,896,7186,4070,430,9291,6,964,249102,887,329,341913
7,Recommendation_manhattan,241368,207,423,2126,563,20665,8,525,72025,3019,984,341913
8,Recommendation_manhattan,222171,542,2206,5992,815,26794,2,702,82145,348,196,341913
9,Recommendation_manhattan,226790,452,2360,5999,1193,22582,1,435,81588,344,169,341913
